In [1]:
from boknis_eck import BEDatabaseWizard
from spectrum import SfgAverager, DummyPlotter 

from datetime import datetime, date, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib.lines import Line2D

%matplotlib qt
plt.style.use("qt.mpltstyle")

bz = BEDatabaseWizard()



In [5]:
%matplotlib qt
plt.style.use("qt.mpltstyle")

temp = bz.df[["sampling_date", "sml_coverage", "bulk_coverage", "one_coverage"]]
BEDatabaseWizard.convert_to_origin_date(temp)
temp.to_csv("onemeter.csv")

C:\Users\lange\Desktop\CharmingSFG\SFG\boknis_eck.py:516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sampling_date'] = df["sampling_date"].dt.strftime('%d.%m.%Y')


In [3]:
temp

,sampling_date,sml_coverage,bulk_coverage,one_coverage
0,2008-10-27,0.195588,NaN,0.132529
1,2008-11-17,0.065895,0.024283,0.025113
2,2008-12-08,NaN,NaN,NaN
3,2009-01-19,NaN,NaN,NaN
4,2009-03-02,0.338041,0.000000,0.079996
5,2009-04-24,0.212656,0.052226,0.276638
6,2009-05-20,NaN,NaN,NaN
7,2009-06-23,0.319091,0.143634,0.063238
8,2009-07-14,0.497407,0.000000,0.067602
9,2009-08-20,NaN,0.000000,NaN
